In [1]:
import pandas as pd
import torch
import pickle

from bs4 import BeautifulSoup

import seaborn as sns
import torch.nn as nn
import numpy as np

In [2]:
df = pd.read_csv("Resume.csv")

In [3]:
df.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/agniksaha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/agniksaha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import regex as re
import string
import numpy as np
import nltk.data
import re
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, sent_tokenize, pos_tag

[nltk_data] Downloading package punkt to /Users/agniksaha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/agniksaha/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [6]:
def convert_to_lower(text):
    return text.lower()

def remove_numbers(text):
    text = re.sub(r'd+' , '', text)
    return text

def remove_white_space(text):
    text = text.strip()
#      text = text.encode('utf-8', errors='ignore').decode('utf-8')
    return text


In [7]:
def preprocess_text(text):
    text = remove_numbers(text)
    text = convert_to_lower(text)
    text = remove_white_space(text)
    return text


In [8]:
df["Resume_html_str"] = df['Resume_html'].apply(lambda x: BeautifulSoup(x).get_text())

In [9]:
df["Resume_html_str"] = df['Resume_html_str'].apply(lambda x: preprocess_text(x))

In [14]:
from keyphrase_vectorizers import KeyphraseCountVectorizer
from keybert import KeyBERT
def get_keyword_bert(docs):
    try:
        vectorizer = KeyphraseCountVectorizer()
        kw_model = KeyBERT()
        onewords = kw_model.extract_keywords(docs=docs, vectorizer=vectorizer,top_n=5,stop_words='english')
        return onewords
    except:
        pass        

In [16]:
# !pip install multi_rake
from multi_rake import Rake

def get_keywords_rake(full_text):
    try:
        rake = Rake()
        keywords = rake.apply(full_text)
        return keywords
    except:
        pass

In [15]:
df['keywords_bert'] = df.Resume_html_str.apply(lambda x: get_keyword_bert(x))

In [17]:
df['keywords_rake'] = df.Resume_html_str.apply(lambda x: get_keywords_rake(x))

In [18]:
df.to_csv("filtered_resume.csv")

In [19]:
df

,ID,Resume_str,Resume_html,Category,Resume_html_str,keywords_bert,keywords_rake
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr administrator/marketing associate\n\nhr adm...,"[(hr administrator, 0.5612), (missouri dot sup...","[(micros opera pms, 9.0), (hospital stays prio..."
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR,"hr specialist, us hr operations summary ...","[(us hr operations, 0.6432), (hr managers, 0.6...","[(hire benefits briefing, 9.0), (etermine prog..."
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr director summary over 20 years e...,"[(hris development, 0.4957), (human resource p...","[(friens university topeka, 9.0), (leaing chan..."
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR,"hr specialist summary deicate, driven...","[(customer service expertise, 0.5182), (hr spe...","[(soli team player, 9.0), (answer questions re..."
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr manager skill highlights ...,"[(hr skills, 0.7018), (human resource manageme...","[(workforce planning initiatives, 9.0), (overs..."
...,...,...,...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION,rank: sgt/e-5 non- commissioned officer in cha...,"[(4th combat aviation brigae mail room, 0.5415...","[(managing inventory control, 9.0), (excellent..."
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION,"government relations, communications and organ...","[(leas external communications team leverage, ...","[(resourceful * successful, 9.0), (develops st..."
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION,geek squad agent professional profile ...,"[(geek squad agent, 0.5774), (technical suppor...","[(teste tanks accoring, 9.0), (information tec..."
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION,program director / office manager su...,"[(office manager, 0.5302), (program director, ...","[(tech savvy professional, 9.0), (social meia ..."


In [2]:
df1 = pd.read_csv("filtered_resume.csv")
df2 = pd.read_csv("resumes_from_pdf_2.csv")

In [3]:
df1 = df1[["keywords_bert","Category"]]
df2 = df2[["keywords_bert","label"]]

In [4]:
df1.rename(columns = {'Category':'label'}, inplace = True)

In [5]:
df = pd.concat([df1, df2])

In [7]:
len(df)

4968

In [6]:
df['label'].value_counts()

INFORMATION-TECHNOLOGY    240
BUSINESS-DEVELOPMENT      240
FINANCE                   236
ADVOCATE                  236
ACCOUNTANT                236
ENGINEERING               236
CHEF                      236
AVIATION                  234
FITNESS                   234
SALES                     232
BANKING                   230
HEALTHCARE                230
CONSULTANT                230
CONSTRUCTION              224
PUBLIC-RELATIONS          222
HR                        220
DESIGNER                  214
ARTS                      206
TEACHER                   204
APPAREL                   194
DIGITAL-MEDIA             192
AGRICULTURE               126
AUTOMOBILE                 72
BPO                        44
Name: label, dtype: int64

In [8]:
from ast import literal_eval
df.keywords_bert = df.keywords_bert.apply(literal_eval)

In [9]:
def get_first_ele(x):
    res = []
    for i in x:
        res.append(i[0])
    return ','.join(res)

df['keywords_bert'] = df.keywords_bert.apply(lambda x: get_first_ele(x))

In [10]:
df['keywords_bert']

0       hr administrator,missouri dot supervisor train...
1       us hr operations,hr managers,us hr operations ...
2       hris development,human resource program,human ...
3       customer service expertise,hr specialist,custo...
4       hr skills,human resource management,hr service...
                              ...                        
2479    automobile liability claims representative,gen...
2480    automobile transporter,car transporter,car hau...
2481    programming,statistics softwares,mechanical sk...
2482    customer relations specialist,customer service...
2483    senior fraud claims specialist degree,certifie...
Name: keywords_bert, Length: 4968, dtype: object

In [11]:
df.head()

,keywords_bert,label
0,"hr administrator,missouri dot supervisor train...",HR
1,"us hr operations,hr managers,us hr operations ...",HR
2,"hris development,human resource program,human ...",HR
3,"customer service expertise,hr specialist,custo...",HR
4,"hr skills,human resource management,hr service...",HR
